In [ ]:
wc0mXwqMhCSKygQu1GIWqG8G
POlNbqIc3kHIdrwGvXBP3X6HIkB8hfAg

In [ ]:
import sys
import json
import base64


IS_PY3 = sys.version_info.major == 3
if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    from urllib.parse import quote_plus
else:
    import urllib2
    from urllib import quote_plus
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode

# 防止https证书校验不正确
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

API_KEY = 'BiMSrevGS9n1gLEE168OMwFW'#'wc0mXwqMhCSKygQu1GIWqG8G'

SECRET_KEY = 'hqozg0ZvA5IgR6kSGQTU0QvMnzb1bonh'#'POlNbqIc3kHIdrwGvXBP3X6HIkB8hfAg'


OCR_URL = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate_basic"
#OCR_URL = "https://aip.baidubce.com/rest/2.0/ocr/v1/form"  #TABLE VERSION
#OCR_URL = 'https://aip.baidubce.com/rest/2.0/solution/v1/form_ocr/request'
#https://aip.baidubce.com/rest/2.0/solution/v1/form_ocr/request

#https://aip.baidubce.com/rest/2.0/solution/v1/form_ocr/get_request_result

"""  TOKEN start """
TOKEN_URL = 'https://aip.baidubce.com/oauth/2.0/token'


"""
    获取token
"""
def fetch_token():
    params = {'grant_type': 'client_credentials',
              'client_id': API_KEY,
              'client_secret': SECRET_KEY}
    post_data = urlencode(params)
    if (IS_PY3):
        post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req, timeout=5)
        result_str = f.read()
    except URLError as err:
        print(err)
    if (IS_PY3):
        result_str = result_str.decode()


    result = json.loads(result_str)

    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if not 'brain_all_scope' in result['scope'].split(' '):
            print ('please ensure has check the  ability')
            exit()
        return result['access_token']
    else:
        print ('please overwrite the correct API_KEY and SECRET_KEY')
        exit()

"""
    读取文件
"""
def read_file(image_path):
    f = None
    try:
        f = open(image_path, 'rb')
        return f.read()
    except:
        print('read image file fail')
        return None
    finally:
        if f:
            f.close()


"""
    调用远程服务
"""
def request(url, data):
    req = Request(url, data.encode('utf-8'))
    has_error = False
    try:
        f = urlopen(req)
        result_str = f.read()
        if (IS_PY3):
            result_str = result_str.decode()
        return result_str
    except  URLError as err:
        print(err)

In [ ]:
# 获取access token
token = fetch_token()

# 拼接通用文字识别高精度url
image_url = OCR_URL + "?access_token=" + token

text = ""

# 读取书籍页面图片
file_content = read_file('pngs/1.png')

# 调用文字识别服务
result = request(image_url, urlencode({'image': base64.b64encode(file_content)}))

# 解析返回结果
#result_json = json.loads(result)
#for words_result in result_json["words_result"]:
#    text = text + words_result["words"]

# 打印文字
#print(text)

In [ ]:
import requests
import base64
import time


request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/form"
# 二进制方式打开图片文件
f = open('pngs/1.png', 'rb')
img = base64.b64encode(f.read())

params = {"image":img,"is_sync":"true","request_type":"excel"}
access_token = fetch_token() #'[调用鉴权接口获取的token]'
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
time.sleep(2) 
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())

In [22]:
import os
import sys
import requests
import time
import tkinter as tk
from tkinter import filedialog
from aip import AipOcr
 
 
 
# 定义常量
APP_ID = '23619402'
API_KEY = 'BiMSrevGS9n1gLEE168OMwFW'#'wc0mXwqMhCSKygQu1GIWqG8G'

SECRET_KEY = 'hqozg0ZvA5IgR6kSGQTU0QvMnzb1bonh'#'POlNbqIc3kHIdrwGvXBP3X6HIkB8hfAg'
# 初始化AipFace对象
client = AipOcr(APP_ID, API_KEY, SECRET_KEY)
 
# 读取图片
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()
 
 
#文件下载函数
def file_download(url, file_path):
    r = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(r.content)


In [24]:
root = tk.Tk()
root.withdraw()
data_dir = 'pngs/'
result_dir = 'output/'
num = 0
for name in os.listdir(data_dir):
    print ('{0} : {1} 正在处理：'.format(num+1, name.split('.')[0]))
    image = get_file_content(os.path.join(data_dir, name))
    res = client.tableRecognitionAsync(image)
    # print ("res:", res)
    if 'error_code' in res.keys():
        print ('Error! error_code: ', res['error_code'])
        sys.exit()
    req_id = res['result'][0]['request_id']    #获取识别ID号

    for count in range(1, 20):    #OCR识别也需要一定时间，设定10秒内每隔1秒查询一次
        res = client.getTableRecognitionResult(req_id)    #通过ID获取表格文件XLS地址
        print(res['result']['ret_msg'])
        if res['result']['ret_msg'] == '已完成':
            break    #云端处理完毕，成功获取表格文件下载地址，跳出循环
        else:
            time.sleep(1)

    url = res['result']['result_data']
    xls_name = name.split('.')[0] + '.xls'
    file_download(url, os.path.join(result_dir, xls_name))
    num += 1
    print ('{0} : {1} 下载完成。'.format(num, xls_name))
    time.sleep(1)

1 : 1 正在处理：
进行中
已完成
1 : 1.xls 下载完成。
